In [42]:
import pandas as pd
import plotly_express as px

In [2]:
vaccine_data = pd.read_excel('../Labb/Data/Folkhalsomyndigheten_Covid19_Vaccine.xlsx',
                              sheet_name='Vaccinerade kommun och ålder')
vaccine_data.head()

,Län,Län_namn,Kommun,Kommun_namn,Ålder,Befolkning,Antal minst 1 dos,Antal minst 2 doser,Antal 3 doser,Antal 4 doser,Andel minst 1 dos,Andel minst 2 doser,Andel 3 doser,Andel 4 doser
0,1,Stockholms län,114,Upplands Väsby,12-15,2422,1206,1046,NaN,NaN,0.497936,0.431874,NaN,NaN
1,1,Stockholms län,114,Upplands Väsby,16-17,1203,839,755,NaN,NaN,0.697423,0.627598,NaN,NaN
2,1,Stockholms län,114,Upplands Väsby,18-29,6692,4887,4469,1959.0,NaN,0.730275,0.667812,0.292738,NaN
3,1,Stockholms län,114,Upplands Väsby,30-39,7332,5542,5240,2878.0,NaN,0.755865,0.714675,0.392526,NaN
4,1,Stockholms län,114,Upplands Väsby,40-49,6946,5592,5429,3719.0,NaN,0.805068,0.781601,0.535416,NaN


## Uppgift 2 - uppvärming vaccindata

a) Hur många län finns representerade i datasetet?
b) Hur många kommuner finns representerade i datasetet?
c) Hur stor är befolkningen som är representerad i datasetet?

In [16]:
# a)
print("Antal län representerade i datasetet:", vaccine_data['Län'].nunique(), "län")

# b)
print("Antal kommuner representerade i datasetet:", vaccine_data['Kommun'].nunique(), "kommuner")

# c)
print("Storlek på befolkning som är representerad i datasetet:",vaccine_data['Befolkning'].sum(), "st")

Antal län representerade i datasetet: 21 län
Antal kommuner representerade i datasetet: 290 kommuner
Storlek på befolkning som är representerad i datasetet: 9092790 st


d) Beräkna hur många barn under 18 år det finns i Sverige baserat på det här datasetet. Du får leta upp
statistik på hur stor totala befolkningen är i Sverige.

Svar:

Baserat på datasetet och den senast rapporterade totala befolkningen i Sverige (10 545 300), <Br>
beräknas det finnas ungefär **864,438** barn under 18 år i Sverige.

In [41]:
barn_grupper = ['12-15', '16-17']

barn_befolkning_dataset = vaccine_data[vaccine_data['Ålder'].isin(barn_grupper)]['Befolkning'].sum()

# Statistik på befolkning i Sverige https://www.scb.se/hitta-statistik/statistik-efter-amne/befolkning/befolkningens-sammansattning/befolkningsstatistik/
# Vid halvårsskiftet 2023 var 10 545 300 personer folkbokförda i Sverige.

total_befolkning = vaccine_data['Befolkning'].sum() 
total_befolkning_sverige = 10545300
andel_barn_i_dataset = barn_befolkning_dataset / total_befolkning
barn_under_18_sverige = andel_barn_i_dataset * total_befolkning_sverige

barn_under_18_sverige.round()

864438.0

## Uppgift 2
e) Rita ett diagram som visar på åldersfördelningen av Sveriges befolkning.

In [45]:
åldersfördelning = vaccine_data.groupby('Ålder')['Befolkning'].sum().reset_index()
åldersfördelning

,Ålder,Befolkning
0,12-15,503831
1,16-17,241539
2,18-29,1475950
3,30-39,1467590
4,40-49,1298156
5,50-59,1339798
6,60-69,1121922
7,70-79,1033113
8,80-89,496750
9,90 eller äldre,114141


In [47]:
fig_ålderfördelning = px.bar(åldersfördelning, x='Ålder', y='Befolkning',
                             title='Åldersfördelning av Sveriges befolkning',
                             labels={'Befolkning': 'Antal personer'})
fig_ålderfördelning.show()
fig_ålderfördelning.write_html("../Labb/Visualiseringar/Uppgift_2/Åldersfördelning_sverige.html")

f) Rita stapeldiagram för **andel med minst 1 dos per län**, **andel två doser per län** och **andel tre doser per
län.**

In [112]:
# Beräkna totalt antal personer för varje doskategori i alla län
totalt_per_doskategori_alla_län = vaccine_data.groupby('Län_namn').agg({
    'Befolkning': 'sum',  # Total befolkning i varje län
    'Antal minst 1 dos': 'sum',
    'Antal minst 2 doser': 'sum',
    'Antal 3 doser': 'sum',
}).reset_index()

# Beräkna andel vaccinerade för varje doskategori
totalt_per_doskategori_alla_län['Andel minst 1 dos'] = totalt_per_doskategori_alla_län['Antal minst 1 dos'] / totalt_per_doskategori_alla_län['Befolkning']
totalt_per_doskategori_alla_län['Andel minst 2 doser'] = totalt_per_doskategori_alla_län['Antal minst 2 doser'] / totalt_per_doskategori_alla_län['Befolkning']
totalt_per_doskategori_alla_län['Andel 3 doser'] = totalt_per_doskategori_alla_län['Antal 3 doser'] / totalt_per_doskategori_alla_län['Befolkning']

# Konvertera andelarna till procent
totalt_per_doskategori_alla_län[['Andel minst 1 dos', 'Andel minst 2 doser', 'Andel 3 doser']] *= 100

totalt_per_doskategori_alla_län[['Län_namn', 'Andel minst 1 dos', 'Andel minst 2 doser', 'Andel 3 doser']].head()  # Visa bara de första raderna för överblick


# Skapa stapeldiagram för alla län
fig_alla_län = px.bar(
    totalt_per_doskategori_alla_län, 
    x='Län_namn', 
    y=['Andel minst 1 dos', 'Andel minst 2 doser', 'Andel 3 doser'],
    title='Andel vaccinerade i alla län i Sverige (i procent)',
    labels={'value': 'Andel (%)', 'variable': 'Dos Kategori', 'Län_namn': 'Län'},
    barmode='group'
)

# Visa diagrammet
fig_alla_län.show()

fig_alla_län.write_html("../Labb/Visualiseringar/Uppgift_2/Andel_vaccinerade_doser_alla_län.html")

g) Tag **Västra Götalands** län och **Stockholms** län och rita diagram över andelen vaccinerade med minst 1 dos,
två doser, tre doser och fyra doser.

In [114]:
# Filtrera ut data för Västra Götalands län och Stockholms län
län_filter = ['Västra Götalands län', 'Stockholms län']
vaccination_län = vaccine_data[vaccine_data['Län_namn'].isin(län_filter)]

# Beräkna totalt antal personer för varje doskategori i de två länen
totalt_per_doskategori = vaccination_län.groupby('Län_namn').agg({
    'Befolkning': 'sum',  # Total befolkning i varje län
    'Antal minst 1 dos': 'sum',
    'Antal minst 2 doser': 'sum',
    'Antal 3 doser': 'sum',
    'Antal 4 doser': 'sum'
}).reset_index()

# Beräkna andel vaccinerade för varje doskategori
totalt_per_doskategori['Andel minst 1 dos'] = totalt_per_doskategori['Antal minst 1 dos'] / totalt_per_doskategori['Befolkning']
totalt_per_doskategori['Andel minst 2 doser'] = totalt_per_doskategori['Antal minst 2 doser'] / totalt_per_doskategori['Befolkning']
totalt_per_doskategori['Andel 3 doser'] = totalt_per_doskategori['Antal 3 doser'] / totalt_per_doskategori['Befolkning']
totalt_per_doskategori['Andel 4 doser'] = totalt_per_doskategori['Antal 4 doser'] / totalt_per_doskategori['Befolkning']

# Konvertera andelarna till procent
totalt_per_doskategori[['Andel minst 1 dos', 'Andel minst 2 doser', 'Andel 3 doser', 'Andel 4 doser']] *= 100

totalt_per_doskategori[['Län_namn', 'Andel minst 1 dos', 'Andel minst 2 doser', 'Andel 3 doser', 'Andel 4 doser']]


,Län_namn,Andel minst 1 dos,Andel minst 2 doser,Andel 3 doser,Andel 4 doser
0,Stockholms län,83.119900,80.107613,56.946200,11.266703
1,Västra Götalands län,85.507702,83.360064,58.092522,14.040763


In [115]:
fig_andel = px.bar(totalt_per_doskategori,
                                         x='Län_namn',
                                         y=['Andel minst 1 dos', 'Andel minst 2 doser', 'Andel 3 doser', 'Andel 4 doser'],
                                         title='Andel vaccinerade med antal doser i Västra Götaland och Stockholm',
                                         labels={'value': 'Andel', 'variable':'Dos Kategori'},
                                         barmode='group')

fig_andel.show()

fig_andel.write_html("../Labb/Visualiseringar/Uppgift_2/Andel_vaccinerade_doser_sthlm_vsg.html")